In [ ]:
%cd ..

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import collections
import glob
import functools
import logging
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import quantities as pq

import epych
from epych.statistics import alignment

In [ ]:
%matplotlib inline

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
CONDITIONS = ["go_gloexp", "go_seqctl", "lo_gloexp", "lonaive", "lo_rndctl", "igo_seqctl"]
PRETRIAL_SECONDS = 0.5
POSTTRIAL_SECONDS = 0.5

In [ ]:
NWB_FILES = glob.glob('/mnt/data/*.nwb')

In [ ]:
PILOT_FILES = []

In [ ]:
evoked = {}

In [ ]:
aligner = epych.statistics.alignment.AlignmentSummary.unpickle("/mnt/data/000253/visual_alignment")

In [ ]:
ODDBALL_ONSET = 0
ODDBALL_OFFSET = 0.5
events = {
    "Onset": (ODDBALL_ONSET, 'lightgreen'),
    "Offset": (ODDBALL_OFFSET, 'red'),
}

In [ ]:
avgs = {cond: {} for cond in CONDITIONS}

In [ ]:
for cond in CONDITIONS:
    avgs[cond] = epych.recording.EvokedSampling.unpickle("/mnt/data/000253/grandavg_%s" % cond)
    for sig in avgs[cond].signals.values():
        sig._data = (sig.data * pq.units.volt).rescale('mV')
        sig._timestamps = sig._timestamps * pq.second
    logging.info("Loaded grand-averaged LFPs for %s" % cond)

In [ ]:
area_titles = {
    "VISal": "AL",
    "VISam": "AM",
    "VISl": "LM",
    "VISp": "V1",
    "VISpm": "PM",
    "VISrl": "RL",
}
def sigtitle(name, signal):
    if name in area_titles:
        return area_titles[name]
    return name

In [ ]:
for cond in CONDITIONS:
    avgs[cond].plot_signals(cond + '/lfp/', sigtitle=sigtitle, **events)

    csd = avgs[cond].smap(lambda sig: sig.downsample(4).current_source_density(depth_column="vertical"))
    for sig in csd.signals.values():
        sig._data = sig.data.rescale('nA / mm**2')
    csd.plot_signals(cond + '/csd/', cmap='jet_r', sigtitle=sigtitle, **events)
    logging.info("Plotted grand-averaged LFPs, CSDs for condition %s" % cond)